In [112]:
import numpy as np
import pandas as pd
import math 

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis , QuadraticDiscriminantAnalysis

from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor 
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import  Normalizer , scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score

from sklearn.preprocessing import MinMaxScaler , StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  

In [87]:
df = pd.read_csv("data/train.csv")
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.01,Premium,G,VS1,62.7,58.0,6.36,6.31,3.97,8.754
1,1,0.60,Good,G,VS2,64.3,57.0,5.31,5.38,3.43,7.492
2,2,1.60,Premium,H,SI2,59.4,59.0,7.55,7.60,4.50,9.010
3,3,0.30,Very Good,J,VS1,63.5,58.0,4.27,4.24,2.70,6.227
4,4,1.00,Very Good,F,SI2,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.51,Ideal,E,VS2,62.5,55.0,5.08,5.13,3.19,7.412
40451,40451,0.80,Ideal,H,VS2,62.1,54.0,5.96,5.99,3.71,8.016
40452,40452,1.53,Premium,G,SI2,58.7,59.0,7.51,7.45,4.39,9.070
40453,40453,1.51,Ideal,I,VS2,61.6,54.0,7.38,7.42,4.57,9.199


In [88]:
clarity = {
    "IF": 4,
    "VVS1": 3.5,
    "VVS2": 3,
    "VS1": 2.5,
    "VS2": 2,
    "SI1": 1.5,
    "SI2": 1,
    "I1": .5
}
df["clarity"] = df.clarity.map(clarity)
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.01,Premium,G,2.5,62.7,58.0,6.36,6.31,3.97,8.754
1,1,0.60,Good,G,2.0,64.3,57.0,5.31,5.38,3.43,7.492
2,2,1.60,Premium,H,1.0,59.4,59.0,7.55,7.60,4.50,9.010
3,3,0.30,Very Good,J,2.5,63.5,58.0,4.27,4.24,2.70,6.227
4,4,1.00,Very Good,F,1.0,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.51,Ideal,E,2.0,62.5,55.0,5.08,5.13,3.19,7.412
40451,40451,0.80,Ideal,H,2.0,62.1,54.0,5.96,5.99,3.71,8.016
40452,40452,1.53,Premium,G,1.0,58.7,59.0,7.51,7.45,4.39,9.070
40453,40453,1.51,Ideal,I,2.0,61.6,54.0,7.38,7.42,4.57,9.199


In [89]:
cut = {
    "Ideal": 2.5,
    "Premium": 2,
    "Very Good": 1.5,
    "Good": 1,
    "Fair": .5
}
df["cut"] = df.cut.map(cut)
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.01,2.0,G,2.5,62.7,58.0,6.36,6.31,3.97,8.754
1,1,0.60,1.0,G,2.0,64.3,57.0,5.31,5.38,3.43,7.492
2,2,1.60,2.0,H,1.0,59.4,59.0,7.55,7.60,4.50,9.010
3,3,0.30,1.5,J,2.5,63.5,58.0,4.27,4.24,2.70,6.227
4,4,1.00,1.5,F,1.0,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.51,2.5,E,2.0,62.5,55.0,5.08,5.13,3.19,7.412
40451,40451,0.80,2.5,H,2.0,62.1,54.0,5.96,5.99,3.71,8.016
40452,40452,1.53,2.0,G,1.0,58.7,59.0,7.51,7.45,4.39,9.070
40453,40453,1.51,2.5,I,2.0,61.6,54.0,7.38,7.42,4.57,9.199


In [90]:
color = {
    "D": 3.5,
    "E": 3,
    "F": 2.5,
    "G": 2,
    "H": 1.5,
    "I": 1,
    "J": .5
}
df["color"] = df.color.map(color)
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.01,2.0,2.0,2.5,62.7,58.0,6.36,6.31,3.97,8.754
1,1,0.60,1.0,2.0,2.0,64.3,57.0,5.31,5.38,3.43,7.492
2,2,1.60,2.0,1.5,1.0,59.4,59.0,7.55,7.60,4.50,9.010
3,3,0.30,1.5,0.5,2.5,63.5,58.0,4.27,4.24,2.70,6.227
4,4,1.00,1.5,2.5,1.0,62.8,57.0,6.37,6.44,4.02,8.314
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.51,2.5,3.0,2.0,62.5,55.0,5.08,5.13,3.19,7.412
40451,40451,0.80,2.5,1.5,2.0,62.1,54.0,5.96,5.99,3.71,8.016
40452,40452,1.53,2.0,2.0,1.0,58.7,59.0,7.51,7.45,4.39,9.070
40453,40453,1.51,2.5,1.0,2.0,61.6,54.0,7.38,7.42,4.57,9.199


In [91]:
df = df[(df[['x','y','z']] != 0).all(axis=1)]
df.loc[(df['x']==0) | (df['y']==0) | (df['z']==0)]


,id,carat,cut,color,clarity,depth,table,x,y,z,price


In [92]:
#df['volume'] = df['x']*df['y']*df['z']
df.drop(['id','x','y','z'], axis=1, inplace= True)

df.head()

/home/fran/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,carat,cut,color,clarity,depth,table,price
0,1.01,2.0,2.0,2.5,62.7,58.0,8.754
1,0.60,1.0,2.0,2.0,64.3,57.0,7.492
2,1.60,2.0,1.5,1.0,59.4,59.0,9.010
3,0.30,1.5,0.5,2.5,63.5,58.0,6.227
4,1.00,1.5,2.5,1.0,62.8,57.0,8.314


In [93]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=666)

In [94]:
model = RandomForestRegressor()

params = {'n_estimators': [100, 300],
          'max_features': ['sqrt', 0.5, None],
          'max_depth': [10, 30, 100, None],
          'min_samples_leaf': [1, 3, 10]}

grid_search = GridSearchCV(model, param_grid=params, verbose=1, n_jobs=-1,cv=5)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 30, 100, None],
                         'max_features': ['sqrt', 0.5, None],
                         'min_samples_leaf': [1, 3, 10],
                         'n_estimators': [100, 300]},
             verbose=1)

In [95]:
best = grid_search.best_score_
best

0.9897446632066581

In [100]:
best_rf = grid_search.best_estimator_
best_rf

RandomForestRegressor(max_depth=100, max_features=0.5, min_samples_leaf=3,
                      n_estimators=300)

In [101]:
best_rf.score(X, y)


0.9935040167904597

In [102]:
dftest = pd.read_csv("data/test.csv")
dftest

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,2.01,Ideal,H,SI1,61.9,57.0,8.14,8.05,5.01
1,1,0.49,Good,D,VS1,57.5,60.0,5.18,5.25,3.00
2,2,1.03,Premium,F,SI1,58.6,62.0,6.65,6.60,3.88
3,3,0.90,Very Good,E,SI1,63.0,56.0,6.11,6.15,3.86
4,4,0.59,Ideal,D,SI1,62.5,55.0,5.35,5.40,3.36
...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.41,Good,F,I1,63.8,57.0,4.72,4.69,3.00
13481,13481,2.05,Very Good,H,VS2,63.5,62.0,8.05,7.91,5.07
13482,13482,2.22,Premium,I,VS2,60.5,59.0,8.41,8.37,5.08
13483,13483,0.51,Ideal,E,SI2,58.3,62.0,5.20,5.19,3.04


In [103]:
dftest.cut.unique()

array(['Ideal', 'Good', 'Premium', 'Very Good', 'Fair'], dtype=object)

In [104]:
dftest.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
dtype: int64

In [105]:
dftest["cut"] = dftest.cut.map(cut)
dftest["clarity"] = dftest.clarity.map(clarity)
dftest["color"] = dftest.color.map(color)


In [106]:
dftest.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
dtype: int64

In [107]:
#dftest['volume'] = dftest['x']*dftest['y']*dftest['z']
dftest.drop(['id','x','y','z'], axis=1, inplace= True)


In [108]:
dftest["price"] = best_rf.predict(dftest)
dftest

,carat,cut,color,clarity,depth,table,price
0,2.01,2.5,1.5,1.5,61.9,57.0,9.748341
1,0.49,1.0,3.5,2.5,57.5,60.0,7.305119
2,1.03,2.0,2.5,1.5,58.6,62.0,8.501003
3,0.90,1.5,3.0,1.5,63.0,56.0,8.338120
4,0.59,2.5,3.5,1.5,62.5,55.0,7.487980
...,...,...,...,...,...,...,...
13480,0.41,1.0,2.5,0.5,63.8,57.0,6.400930
13481,2.05,1.5,1.5,2.0,63.5,62.0,9.672445
13482,2.22,2.0,1.0,2.0,60.5,59.0,9.749706
13483,0.51,2.5,3.0,1.0,58.3,62.0,7.083259


In [109]:
dfsubmit = dftest[["price"]]
dfsubmit.index.name = 'id'

In [110]:
dfsubmit

,price
id,
0,9.748341
1,7.305119
2,8.501003
3,8.338120
4,7.487980
...,...
13480,6.400930
13481,9.672445
13482,9.749706


In [111]:
dfsubmit.to_csv("submit.csv", index = True)